### Homework: 尝试扩展 GitHubClient，使其支持 Until 参数条件筛选

In [5]:
import os
import requests
import datetime
from loguru import logger
import sys
# Configure Loguru
logger.remove()  # Remove the default logger
logger.add(sys.stdout, level="DEBUG", format="{time} {level} {message}", colorize=True)
logger.add("logs/app.log", rotation="1 MB", level="DEBUG")
# Alias the logger for easier import
LOG = logger
# Make the logger available for import with the alias
__all__ = ["LOG"]
# src/github_client.py
import requests  # 导入requests库用于HTTP请求
from datetime import datetime, date, timedelta  # 导入日期处理模块
import os  # 导入os模块用于文件和目录操作
# from logger import LOG  # 导入日志模块（演示时直接导入）

class GitHubClient:
    def __init__(self, token):
        self.token = token
        self.headers = {'Authorization': f'token {self.token}'}

    def fetch_updates(self, repo, since=None, until=None):
        """
        获取指定仓库在给定时间范围内的更新
        
        Args:
            repo (str): 仓库名称 (格式: "owner/repo")
            since (str): 起始时间 (ISO 8601格式)
            until (str): 结束时间 (ISO 8601格式)
        """
        try:
            updates = {
                'commits': self.fetch_commits(repo, since, until),
                'issues': self.fetch_issues(repo, since, until),
                'pull_requests': self.fetch_pull_requests(repo, since, until)
            }
            LOG.info(f"Successfully fetched updates for {repo} from {since} to {until}")
            return updates
        except Exception as e:
            LOG.error(f"Error fetching updates for {repo}: {str(e)}")
            raise

    def _filter_by_date(self, items, until_date):
        """
        根据until日期过滤项目列表
        
        Args:
            items (list): 要过滤的项目列表
            until_date (str): 结束日期 (ISO 8601格式)
        """
        if not until_date:
            return items
            
        until_datetime = datetime.fromisoformat(until_date.replace('Z', '+00:00'))
        return [
            item for item in items 
            if datetime.fromisoformat(item.get('created_at', '').replace('Z', '+00:00')) <= until_datetime
        ]

    def fetch_commits(self, repo, since=None, until=None):
        """获取提交记录"""
        url = f'https://api.github.com/repos/{repo}/commits'
        params = {'per_page': 100}  # 增加每页数量
        
        if since:
            params['since'] = since
        if until:
            params['until'] = until
            
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            commits = response.json()
            LOG.debug(f"Fetched {len(commits)} commits for {repo}")
            return commits
        except Exception as e:
            LOG.error(f"Error fetching commits for {repo}: {str(e)}")
            raise

    def fetch_issues(self, repo, since=None, until=None):
        """获取问题"""
        url = f'https://api.github.com/repos/{repo}/issues'
        params = {
            'state': 'closed',
            'per_page': 100,
            'sort': 'updated',
            'direction': 'desc'
        }
        
        if since:
            params['since'] = since
            
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            issues = response.json()
            
            # 使用until参数过滤
            if until:
                issues = self._filter_by_date(issues, until)
                
            LOG.debug(f"Fetched {len(issues)} issues for {repo}")
            return issues
        except Exception as e:
            LOG.error(f"Error fetching issues for {repo}: {str(e)}")
            raise

    def fetch_pull_requests(self, repo, since=None, until=None):
        """获取拉取请求"""
        url = f'https://api.github.com/repos/{repo}/pulls'
        params = {
            'state': 'closed',
            'per_page': 100,
            'sort': 'updated',
            'direction': 'desc'
        }
        
        if since:
            params['since'] = since
            
        try:
            response = requests.get(url, headers=self.headers, params=params)
            response.raise_for_status()
            pull_requests = response.json()
            
            # 使用until参数过滤
            if until:
                pull_requests = self._filter_by_date(pull_requests, until)
                
            LOG.debug(f"Fetched {len(pull_requests)} pull requests for {repo}")
            return pull_requests
        except Exception as e:
            LOG.error(f"Error fetching pull requests for {repo}: {str(e)}")
            raise

In [8]:
# 使用示例
github_client = GitHubClient(token=os.getenv("GITHUB_TOKEN"))

# 获取特定时间范围的更新
updates = github_client.fetch_updates(
    repo="langchain-ai/langchain",
    since="2024-11-01T00:00:00Z",
    until="2024-11-15T23:59:59Z"
)

# 处理结果
for commit in updates['commits']:
    print(f"Commit: {commit['sha'][:7]} - {commit['commit']['message']}")

for issue in updates['issues']:
    print(f"Issue #{issue['number']}: {issue['title']}")

for pr in updates['pull_requests']:
    print(f"PR #{pr['number']}: {pr['title']}")

2024-11-15T14:38:57.146289+0800 DEBUG Fetched 100 commits for langchain-ai/langchain
2024-11-15T14:38:59.399125+0800 DEBUG Fetched 100 issues for langchain-ai/langchain
2024-11-15T14:39:02.385410+0800 DEBUG Fetched 100 pull requests for langchain-ai/langchain
2024-11-15T14:39:02.387652+0800 INFO Successfully fetched updates for langchain-ai/langchain from 2024-11-01T00:00:00Z to 2024-11-15T23:59:59Z
Commit: ef2dc9e - docs: update "quickstart" tutorial (#28096)

- Update language / add links in places
- De-emphasize output parsers
- remove deployment section
Commit: f122273 - core[patch]: support numpy 2 (#27991)
Commit: cff70c2 - docs: Add hyperlink to immediately show the table at the bottom of th… (#28102)

Added a hyperlink which can be clicked so users can immediately see the
table and find out the various example selector methods
Commit: 4b641f8 - English Update and fixed a duplicate "the" (#27981)

Fixed a duplicate "the" in the documentation and made the documentation
generally 